In [1]:
import sscv 
import mmcv 
import matplotlib.pyplot as plt 
import os.path as osp
import numpy as np

In [2]:
from glob import glob 
from PIL import Image
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [3]:
config_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2021.04.16_deeplabv3plus_r101-d8_769x769_40k_concrete_crack/deeplabv3plus_r101-d8_769x769_40k_concrete_crack.py'
checkpoint_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2021.04.16_deeplabv3plus_r101-d8_769x769_40k_concrete_crack/iter_40000.pth'

In [4]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/soojin/anaconda3/envs/auto-label/lib/python3.8/site-packages/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


Use load_from_local loader


In [10]:
auto_label_dir = "/home/soojin/UOS-SSaS Dropbox/05. Data/06. Auto Learning/"

dataset_name = 'Road Crack'
raw_img_dir_name = '01. Raw Image'
rsz_img_dir_name = '02. Resized Image'
detection_result_dir_name = '01. Detection Result'


dataset_dir = osp.join(auto_label_dir, dataset_name)
raw_img_dir = osp.join(dataset_dir, raw_img_dir_name)
rsz_img_dir = osp.join(dataset_dir, rsz_img_dir_name)
detection_result_dir = osp.join(rsz_img_dir, detection_result_dir_name)

In [11]:
# Find difference btw raw and resized img lists 
raw_img_list = [osp.basename(x) for x in glob(osp.join(raw_img_dir, '*.jpg'))]
rsz_img_list = [osp.basename(x) for x in glob(osp.join(rsz_img_dir, '*.jpg'))]

imgs_to_rsz_list = [img_path for img_path in raw_img_list if img_path not in rsz_img_list]


In [25]:
join(raw_img_dir, '*.jpg')

NameError: name 'join' is not defined

In [12]:
# Move image from raw folder to resize folder 
for idx, img_filename in enumerate(imgs_to_rsz_list): 
    org_img_path = osp.join(raw_img_dir, img_filename)
    img = sscv.imread(org_img_path)
    rsz_img_path = osp.join(rsz_img_dir, img_filename)
    
    is_height_longer = img.shape[0] > img.shape[1]
    
    if is_height_longer : 
        rsz_img = sscv.resize(img, height = 1024)
    else : 
        rsz_img = sscv.resize(img, width = 1024)
        
    print("Image Num: {}, Original Image Shape: {}, Resized Image Shape : {}, Image Filename: {} "
          .format(idx, img.shape, rsz_img.shape, img_filename)
         )
    
    sscv.imwrite(rsz_img_path, rsz_img)
    clear_output(wait=True)

    

In [13]:
rsz_img_list = [osp.basename(x) for x in glob(osp.join(rsz_img_dir, '*.jpg'))]

for idx, img_filename in enumerate(rsz_img_list): 
    img_filepath = osp.join(rsz_img_dir, img_filename)
    img_basename = osp.basename(img_filename).split('.')[0]
    img = sscv.imread(img_filepath)
    
    result = inference_segmentor(model, img_filepath)
    
    detection_result_with_transparent_background = np.ones(img.shape, dtype = np.uint8)*255 # create white image 
    
    crack_area = result[0] == 1
    detection_result_with_transparent_background[crack_area, 0] =  255
    detection_result_with_transparent_background[crack_area, 1] =  0
    detection_result_with_transparent_background[crack_area, 2] =  0
    
    detection_result_with_transparent_background = Image.fromarray(detection_result_with_transparent_background) 
    detection_result_with_transparent_background = detection_result_with_transparent_background.convert("RGBA")
    
    img_datas = detection_result_with_transparent_background.getdata()
    
    newData = []
    
    for item in img_datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
  
    detection_result_with_transparent_background.putdata(newData)
    detection_result_with_transparent_background.save(osp.join(detection_result_dir, img_basename + '_crack.png'), "PNG")

In [1]:
import cv2
import matplotlib.pyplot as plt
import os.path as osp 
import sscv 

from glob import glob 
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
auto_label_dir = "/run/user/1000/gvfs/afp-volume:host=DiskStation.local,user=shm_nas,volume=Volume_2/02. Training & Test/"

dataset_name = 'mihochun_crack_trainingset'
raw_img_dir_name = '01. raw image'
crop_img_dir_name = '02. cropped images'
label_dir_name = '04. labeling'
pre_label_dir_name = '01. rough-labeling'

dataset_dir = osp.join(auto_label_dir, dataset_name)
raw_img_dir = osp.join(dataset_dir, raw_img_dir_name)
crop_img_dir = osp.join(dataset_dir, crop_img_dir_name)
pre_label_dir = osp.join(crop_img_dir, pre_label_dir_name)

In [5]:
# crop image from raw to cropped image 
from PIL import Image
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette


In [6]:
config_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2021.04.16_deeplabv3plus_r101-d8_769x769_40k_concrete_crack/deeplabv3plus_r101-d8_769x769_40k_concrete_crack.py'
checkpoint_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2021.04.16_deeplabv3plus_r101-d8_769x769_40k_concrete_crack/iter_40000.pth'


In [7]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/soojin/anaconda3/envs/auto-label/lib/python3.8/site-packages/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


Use load_from_local loader
